In [198]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE, KMeansSMOTE, RandomOverSampler, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.combine import SMOTETomek
from ngboost import NGBClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
from collections import Counter

In [199]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")
print(train_df.shape)
#----------drop duplicates for validation-----------------
# train_df = train_df.drop_duplicates()
# train_df = train_df.reset_index(drop=True)

#----------------------------------------------
train_df["WhichTeamWon"]=0
for i in range(len(train_df)):
    if train_df["Team1"][i]==train_df["MatchWinner"][i]:
        train_df["WhichTeamWon"][i]=1
    else:
        train_df["WhichTeamWon"][i]=2
#------------------------------------------------------
catfeatures =[col for col in train_df.columns if type(train_df[col][0])==str]       
for col in catfeatures:
    train_df[col] = train_df[col].astype('category').cat.codes
    test_df[col] = test_df[col].astype('category').cat.codes
months_in_year = 12
# TRAIN
train_df["MonthOfMatch"] = train_df["MonthOfMatch"].astype('category').cat.codes
train_df['sin_month'] = np.sin(2*np.pi*train_df["MonthOfMatch"]/months_in_year)
train_df['cos_month'] = np.cos(2*np.pi*train_df["MonthOfMatch"]/months_in_year)
train_df.drop("MonthOfMatch",axis=1,inplace=True)
#TEST
test_df["MonthOfMatch"] = test_df["MonthOfMatch"].astype('category').cat.codes
test_df['sin_month'] = np.sin(2*np.pi*test_df["MonthOfMatch"]/months_in_year)
test_df['cos_month'] = np.cos(2*np.pi*test_df["MonthOfMatch"]/months_in_year)
test_df.drop("MonthOfMatch",axis=1,inplace=True)
test_df["Team1_Venue"] += 1
test_df["Team2_Venue"] += 1
for i in range(len(test_df)):
    if test_df["Team1_Venue"][i]==1:
        test_df["Team2_Venue"][i]=0
#-----deleting unwanted row-------------
train_df=train_df.drop(1398,axis=0)

#-------------------------------------------------------------
#------------------------------GoodFeatures------------------------------------------
feature = train_df.groupby(['Team1']).agg(
    {'MatchWinner': ['count',lambda x:x.value_counts().index[0]]})
feature.columns = ['Team1win%' + '_'.join(c).strip('_') for c in feature.columns]
train_df = pd.merge(train_df, feature, on = ['Team1'], how='left')
test_df = pd.merge(test_df, feature, on = ['Team1'], how='left')
# --------------------------------who_mostly_wins------------------------------
means = train_df.groupby(['Team1', 'Team2'])["MatchWinner"].agg(['max','min','mean','std'])
means = means.reset_index(level=['Team1', 'Team2'])
train_df = pd.merge(train_df, means, on=['Team1', 'Team2'], how='left',suffixes=('', '_y'))
test_df = pd.merge(test_df, means, on=['Team1', 'Team2'], how='left',suffixes=('', '_y'))
#--------------------------------------------------------------------------------
means = train_df.groupby(['Team1','Team2' ,'Team1_Venue'])["MatchWinner"].agg(['max','mean'])
means = means.reset_index(level=['Team1','Team2', 'Team1_Venue'])
train_df = pd.merge(train_df, means, on=['Team1', 'Team2','Team1_Venue'], how='left',suffixes=('', '_1venue'))
test_df = pd.merge(test_df, means, on=['Team1','Team2', 'Team1_Venue'], how='left',suffixes=('', '_1venue'))
#--------------------------------------------------------------------------------

test_df.isnull().sum()

(2508, 10)


Team1                               0
Team2                               0
Stadium                             0
HostCountry                         0
Team1_Venue                         0
Team2_Venue                         0
Team1_Innings                       0
Team2_Innings                       0
sin_month                           0
cos_month                           0
Team1win%MatchWinner_count          0
Team1win%MatchWinner_<lambda_0>     0
max                                12
min                                12
mean                               12
std                                34
max_1venue                         17
mean_1venue                        17
dtype: int64

In [201]:
train_df

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MatchWinner,WhichTeamWon,sin_month,cos_month,Team1win%MatchWinner_count,Team1win%MatchWinner_<lambda_0>,max,min,mean,std,max_1venue,mean_1venue
0,5,4,37,4,1,0,1,0,4,2,8.660254e-01,5.000000e-01,364,5,5,4,4.518519,0.509175,5,4.518519
1,1,14,84,7,2,2,0,1,1,1,-5.000000e-01,8.660254e-01,386,1,14,1,7.200000,6.543604,14,6.909091
2,9,15,47,9,1,0,0,1,9,1,1.000000e+00,6.123234e-17,253,9,15,9,10.846154,2.882307,15,11.666667
3,7,2,102,6,1,0,0,1,2,2,5.000000e-01,8.660254e-01,50,7,7,2,4.000000,2.738613,7,4.000000
4,6,8,46,5,1,0,0,1,6,1,5.000000e-01,8.660254e-01,49,6,8,6,6.400000,0.894427,6,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,5,15,89,3,2,2,1,0,5,1,1.224647e-16,-1.000000e+00,364,5,15,5,7.068966,4.122508,15,6.875000
2503,1,14,138,0,1,0,0,1,1,1,8.660254e-01,-5.000000e-01,386,1,14,1,7.200000,6.543604,14,7.259259
2504,14,5,135,15,1,0,1,0,14,1,0.000000e+00,1.000000e+00,171,14,14,5,10.192308,4.534483,14,10.192308
2505,15,14,64,16,1,0,0,1,14,2,-1.000000e+00,-1.836970e-16,127,15,15,14,14.200000,0.421637,15,14.200000


In [202]:
features = [col for col in train_df.columns if col not in ("MatchWinner","WhichTeamWon")]
# catfeatures =[col for col in train_df.columns if type(train_df[col][0])==str]
catfeatures =["Team1","Team2","HostCountry","Team1_Innings","Team2_Innings"]
x = train_df[features]
y = train_df["WhichTeamWon"]
test_df = test_df[features]

In [203]:
train_df.dtypes

Team1                                int64
Team2                                int64
Stadium                              int64
HostCountry                          int64
Team1_Venue                           int8
Team2_Venue                           int8
Team1_Innings                         int8
Team2_Innings                         int8
MatchWinner                          int64
WhichTeamWon                         int64
sin_month                          float64
cos_month                          float64
Team1win%MatchWinner_count           int64
Team1win%MatchWinner_<lambda_0>      int64
max                                  int64
min                                  int64
mean                               float64
std                                float64
max_1venue                           int64
mean_1venue                        float64
dtype: object

In [204]:
kfold, scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=27), []
for train_, test_ in kfold.split(x,y):
    x_train, x_test = x.iloc[train_], x.iloc[test_]
    y_train, y_test = y.iloc[train_], y.iloc[test_]
    print(y_test.shape,x_test.shape)
    model = CatBoostClassifier(random_state=27,num_trees=170,
                               cat_features = catfeatures,verbose=500)
    model.fit(x_train,y_train)
    preds = model.predict_proba(x_test)
    
    model = LGBMClassifier(num_leaves=5,colsample_bytree=0.4,learning_rate=0.4,max_depth=3,
                           random_state=27,reg_alpha=0.2,boosting_type='dart')
    model.fit(x_train.values,y_train)
    preds2 = model.predict_proba(x_test.values)
    
    preds = list()
    LGBM_weight=0.3
    GBM_weight=0.6
    for i, j in zip(preds2, preds1):
        xx = [(LGBM_weight * i[0]) + (GBM_weight * j[0]), (LGBM_weight * i[1]) + (GBM_weight * j[1])]
        preds.append(xx)
    
    score = log_loss(y_test, preds)
    scores.append(score)
    print(score)
print(str(model).split("(")[0], sum(scores)/len(scores))

(502,) (502, 18)
Learning rate set to 0.070407
0:	learn: 0.6873669	total: 18.4ms	remaining: 3.1s
169:	learn: 0.5366900	total: 2.79s	remaining: 0us
0.5939491036844212
(502,) (502, 18)
Learning rate set to 0.070407
0:	learn: 0.6874176	total: 20.7ms	remaining: 3.5s
169:	learn: 0.5395925	total: 2.25s	remaining: 0us
0.5858118141955433
(501,) (501, 18)
Learning rate set to 0.070422
0:	learn: 0.6876039	total: 13.9ms	remaining: 2.35s
169:	learn: 0.5300196	total: 2.71s	remaining: 0us
0.6316769512275275
(501,) (501, 18)
Learning rate set to 0.070422
0:	learn: 0.6870121	total: 16.1ms	remaining: 2.73s
169:	learn: 0.5279152	total: 2.63s	remaining: 0us
0.6283866558118295
(501,) (501, 18)
Learning rate set to 0.070422
0:	learn: 0.6880799	total: 18ms	remaining: 3.04s
169:	learn: 0.5396614	total: 2.16s	remaining: 0us
0.6009760035766252
<catboost.core.CatBoostClassifier object at 0x000001AD690CB908> 0.6081601056991893


In [ ]:
#0.6081601056991893 

In [148]:
#LGBM Alone
kfold, scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=27), []
for train_, test_ in kfold.split(x,y):
    x_train, x_test = x.iloc[train_], x.iloc[test_]
    y_train, y_test = y.iloc[train_], y.iloc[test_]
    model = LGBMClassifier(num_leaves=5,colsample_bytree=0.4,learning_rate=0.4,max_depth=3,
                           random_state=27,reg_alpha=0.2,boosting_type='dart')
    model.fit(x_train.values,y_train,categorical_feature=[0,1,3,4,6])
    preds = model.predict_proba(x_test.values)
    score = log_loss(y_test, preds)
    scores.append(score)
    print(score)
print(str(model).split("(")[0], sum(scores)/len(scores))

0.5866584941138662
0.5934651973806399
0.6352923323392936
0.6387955922674496
0.5965620122660299
LGBMClassifier 0.6101547256734559


In [154]:
#XGB Alone
kfold, scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=27), []
for train_, test_ in kfold.split(x,y):
    x_train, x_test = x.iloc[train_], x.iloc[test_]
    y_train, y_test = y.iloc[train_], y.iloc[test_]
    model = XGBClassifier(random_state=27,max_depth=3,n_estimators=25)
    model.fit(x_train.values,y_train)
    preds = model.predict_proba(x_test.values)
    score = log_loss(y_test, preds)
    scores.append(score)
    print(score)
print(str(model).split("(")[0], sum(scores)/len(scores))

0.5957989619777675
0.6005911768425212
0.6331899298871765
0.6351236226650591
0.5958294849396645
XGBClassifier 0.6121066352624378


In [ ]:
#History
#CatBoost:
#without features:0.6138779945839952 / Dropping: 0.6283985786987546
#with Team1bat1stwinning% feature : 0.6098229410370798
#with Team1Venuewinning% feature : 0.6089004455852214
#optimal featutes: 0.608070942985481 / Dropping: 0.6169618127716114
#overTrained features: 0.5935706991924635/ Dropping: 0.6148279721517143
#-----------------
#LGBM:
#without features:0.6356475470788228 / Dropping:0.6474816724454667
#optimal featutes:0.6152302211318373 / Dropping:0.634207974370743
#----------------
#XGB:
#without features:0.6483861383069611 / 0.6502544204819645
#optimal featutes:0.6186337418733754 / Dropping:0.6307907271823614

In [212]:
all_preds = []
for seed_ in np.arange(8, 28):
    print(seed_)
    cat=CatBoostClassifier(random_state=27,num_trees=170,
                               cat_features = catfeatures,verbose=1000)
    cat.fit(x,y)
    all_preds.append(cat.predict_proba(test_df))

8
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 13.3ms	remaining: 2.25s
169:	learn: 0.5348977	total: 2.71s	remaining: 0us
9
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 15.8ms	remaining: 2.68s
169:	learn: 0.5348977	total: 2.95s	remaining: 0us
10
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 14.2ms	remaining: 2.4s
169:	learn: 0.5348977	total: 2.6s	remaining: 0us
11
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 13.8ms	remaining: 2.33s
169:	learn: 0.5348977	total: 2.41s	remaining: 0us
12
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 15.3ms	remaining: 2.58s
169:	learn: 0.5348977	total: 2.51s	remaining: 0us
13
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 13ms	remaining: 2.19s
169:	learn: 0.5348977	total: 2.46s	remaining: 0us
14
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 15.9ms	remaining: 2.68s
169:	learn: 0.5348977	total: 3.38s	remaining: 0us
15
Learning rate set to 0.077455
0:	learn: 0.6865498	total: 22.6ms	r

In [228]:
final_preds = all_preds[0] + all_preds[1] +all_preds[2] + all_preds[3] +all_preds[4] + all_preds[5] +all_preds[6] + all_preds[7] +all_preds[8] + all_preds[9] + all_preds[10] +all_preds[11]+all_preds[12] + all_preds[13] +all_preds[14]+all_preds[15] + all_preds[16] +all_preds[17] +all_preds[18] + all_preds[19] 
final_preds = final_preds/20
final_preds

array([[0.38571096, 0.61428904],
       [0.38145273, 0.61854727],
       [0.4419432 , 0.5580568 ],
       ...,
       [0.52478763, 0.47521237],
       [0.63908516, 0.36091484],
       [0.58005328, 0.41994672]])

In [231]:
# preds=model.predict_proba(test_df)
op = pd.DataFrame(final_preds)
wholepred = pd.DataFrame(np.zeros((1075,16),dtype=float))
for i in range(len(test_df)):
    wholepred[test_df["Team1"][i]][i]=op[0][i]
    wholepred[test_df["Team2"][i]][i]=op[1][i]
wholepred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.000000,0.385711,0.0,0.614289,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.618547,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.381453,0.000000
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.441943,0.558057,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.563023,0.436977,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.000000,0.793267,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.206733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.0,0.000000,0.000000,0.0,0.000000,0.879779,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.120221
1071,0.0,0.602584,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.397416,0.000000,0.000000,0.000000
1072,0.0,0.000000,0.000000,0.0,0.000000,0.524788,0.0,0.0,0.0,0.000000,0.475212,0.0,0.000000,0.000000,0.000000,0.000000
1073,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.639085,0.0,0.000000,0.360915,0.000000,0.000000


In [232]:
wholepred.to_excel("Final.xlsx",index=False)

In [ ]:
#Not worked Features

#---------------------------------------------------
# team1_list = train_df['Team1'].unique()
# team2_list = train_df['Team2'].unique()
# import itertools
# teams_comb_list = list(itertools.product(team1_list, team2_list))

# final_teams_comb_list = [(val, key) for (val, key) in teams_comb_list if key != val]
# train_df['is_train']  = 1
# all_dfs = []
# all_team1_team2_wins_df = pd.DataFrame()
# for i in range(len(final_teams_comb_list)):
#   try:
#     df = train_df[(train_df['Team1'] == final_teams_comb_list[i][0]) & (train_df['Team2'] == final_teams_comb_list[i][1])]
#     gdf = df.groupby(["Team1", "Team2","MatchWinner"]).sum().reset_index()
#     gdf.loc[gdf.MatchWinner == gdf.Team1,'winning_team']='Team1'
#     gdf.loc[gdf.MatchWinner == gdf.Team2, 'winning_team']='Team2'
#     gdf = pd.pivot_table(gdf,index=["Team1","Team2"],values=["is_train"], columns=["winning_team"]).reset_index()
 
#     if (gdf.shape[1] == 4):
#       gdf.columns = ['Team1','Team2','Team1_Wins','Team2_Wins']
#       all_dfs.append(gdf)
#     else:           
#       if ((gdf.loc[:, pd.IndexSlice[:, ['Team1']]]).empty):  
#         gdf.columns = ['Team1','Team2','Team2_Wins']
#         gdf['Team1_Wins'] =0
#         gdf = gdf[['Team1','Team2', 'Team1_Wins','Team2_Wins']]
#         all_dfs.append(gdf)  
#       elif ((gdf.loc[:, pd.IndexSlice[:, ['Team2']]]).empty):      
#         gdf.columns = ['Team1','Team2','Team1_Wins']
#         gdf['Team2_Wins'] =0
#         gdf = gdf[['Team1','Team2', 'Team1_Wins','Team2_Wins']]
#         all_dfs.append(gdf)
#   except:
#     pass
  
#     all_team1_team2_wins_df = pd.concat(all_dfs)
# all_team1_team2_wins_df.drop_duplicates(inplace=True)
# print(all_team1_team2_wins_df.shape)
# all_team1_team2_wins_df.fillna(0,inplace=True)
# all_team1_team2_wins_df['Team1_Win_Percentage_Overall'] = all_team1_team2_wins_df['Team1_Wins']/(all_team1_team2_wins_df['Team1_Wins'] + all_team1_team2_wins_df['Team2_Wins'])
# all_team1_team2_wins_df.head()
# train_df = pd.merge(train_df, all_team1_team2_wins_df, on = ['Team1','Team2'], how='left')
# test_df = pd.merge(test_df, all_team1_team2_wins_df, on = ['Team1','Team2'], how='left')
# train_df.drop("is_train",axis=1,inplace=True)